In [1]:
import numpy as np
import pandas as pd

In [2]:
test_df = pd.read_csv("/content/drive/MyDrive/#Kaggle/titanic/test.csv")
train_df = pd.read_csv("/content/drive/MyDrive/#Kaggle/titanic/train.csv")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
column_label = train_df.columns

#### 전체 column 
승객 아이디, 생존여부, 승선등급, 이름, 성, 나이, 형제자매/배우자, 부모/자식 수, 티켓 ID?, 선실번호, 승선위치 

#### 생각
1. 승객 아이디, 이름, 티켓 아이디는 너무 데이터가 난해해서 필요 없을 거 같음
2. Embarked가 승선위치인데, 3가지의 string으로 구분되어 있음, 이를 one-hot 으로 바꿔서 학습 시켜보면 어떨꽈 
3. Nan 값에 대해서, 필드가 다양하니 단순 평균 말고 다른 columns(feature)에 대해서 최대한 일치하는 것으로 평균을 잡아서 
4. Cabin이 선실을 예약한 사람들의 기록이 있는 사람이라고 생각해서 값이 있으면 1, 없으면 0 으로 하려고 했는데 값이 있는 사람들이 204/891 말이 안되는듯 


##### 삭제 
-> P_Id, Name, Ticket, Cabin 

##### 변형 
-> Embarked 

##### Data Imputation 
-> 필드가 최대한 일치하는 친구들과 함께 평균내서 적용


In [5]:
feature_label = list(train_df.columns)
feature_label.remove("PassengerId")
feature_label.remove("Name")
feature_label.remove("Cabin")
feature_label.remove("Ticket")
feature_label

['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [6]:
selected_df = train_df[feature_label]
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [7]:
# 쉽게 쓰이는거 모음, 리턴은 3개를 다 리턴해줌 
def dataProcessing(srcDataFrame, resetIndex = True, printInfo=False):
  returnList = []

  returnList.append(srcDataFrame.fillna(srcDataFrame.mean())) # selected_df.mean() -> 컬럼 별로 평균 값이 나옴 
  returnList.append(srcDataFrame.fillna(0))
  returnList.append(srcDataFrame.dropna())

  if resetIndex :
    returnList[-1].reset_index(drop=True)

  if printInfo:
    for unit in returnList:
      print(unit)
      unit.info()
      print(unit.mean())
      print("\n\n")
  
  return returnList

In [8]:
processed_df = dataProcessing(selected_df)

In [9]:
def oneHotEncoding(srcDataFrame):
  if type(srcDataFrame) is list :
    
    for idx, unit in enumerate(srcDataFrame):
      srcDataFrame[idx] = pd.get_dummies(unit)

  else :
    srcDataFrame = pd.get_dummies(srcDataFrame)
  
  return srcDataFrame

In [10]:
processed_df = oneHotEncoding(processed_df)

In [11]:
# for validation train 
def trainTestSplit(srcDataframe, yLabel, ratio=0.8):
  from sklearn.model_selection import train_test_split

  # *array -> lists, numpy, pandas dataframe 
  if type(srcDataframe) is list :
    for idx, unit in enumerate(srcDataframe):
      feature_index = list(unit.columns)
      feature_index.remove(yLabel)
      srcDataframe[idx] = train_test_split(unit[feature_index].to_numpy(),unit[yLabel].to_numpy(), train_size=ratio, random_state=411)
  else:

    feature_index = list(srcDataframe.columns)
    feature_index.remove(yLabel)
    srcDataframe = train_test_split(srcDataframe[feature_index].to_numpy(),srcDataframe[yLabel].to_numpy(), train_size=ratio, random_state=411)


  return srcDataframe  


In [12]:
splitted = trainTestSplit(processed_df.copy(), 'Survived')

In [102]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

HBD = 411
for i in range(len(splitted)):
  train_X, test_X, train_y, test_y = splitted[i]

  minmax = MinMaxScaler().fit(train_X)
  Std = StandardScaler().fit(train_X)
  maxAbs = MaxAbsScaler().fit(train_X)
  robust = RobustScaler().fit(train_X)

  minmaxTrain_X = minmax.transform(train_X)
  StdTrain_X = Std.transform(train_X)
  maxAbsTrain_X = maxAbs.transform(train_X)
  robustTrain_X = robust.transform(train_X)

  # minmax = MinMaxScaler().fit(test_X)
  # Std = StandardScaler().fit(test_X)
  # maxAbs = MaxAbsScaler().fit(test_X)
  # robust = RobustScaler().fit(test_X)

  minmaxtest_X = minmax.transform(test_X)
  Stdtest_X = Std.transform(test_X)
  maxAbstest_X = maxAbs.transform(test_X)
  robusttest_X = robust.transform(test_X)


  import numpy as np
  # sklearn test 
  from sklearn.linear_model import PassiveAggressiveClassifier
  pac = PassiveAggressiveClassifier().fit(train_X, train_y)
  print("PassiveAggressiveClassifier", pac.score(test_X, test_y))

  pac = PassiveAggressiveClassifier().fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", pac.score(minmaxtest_X, test_y))
  pac = PassiveAggressiveClassifier().fit(StdTrain_X, train_y)
  print("StandardScaler", pac.score(Stdtest_X, test_y))
  pac = PassiveAggressiveClassifier().fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", pac.score(maxAbstest_X, test_y))
  pac = PassiveAggressiveClassifier().fit(robustTrain_X, train_y)
  print("RobustScaler", pac.score(robusttest_X, test_y))
  print()

  from sklearn.linear_model import RidgeClassifier
  ridge = RidgeClassifier().fit(train_X, train_y)
  print("RidgeClassifier",ridge.score(test_X, test_y))

  ridge = RidgeClassifier().fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", ridge.score(minmaxtest_X, test_y))
  ridge = RidgeClassifier().fit(StdTrain_X, train_y)
  print("StandardScaler", ridge.score(Stdtest_X, test_y))
  ridge = RidgeClassifier().fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", ridge.score(maxAbstest_X, test_y))
  ridge = RidgeClassifier().fit(robustTrain_X, train_y)
  print("RobustScaler", ridge.score(robusttest_X, test_y))
  print()

  from sklearn.linear_model import RidgeClassifierCV
  ridgeCv = RidgeClassifierCV().fit(train_X, train_y)
  print("RidgeClassifierCV",ridgeCv.score(test_X, test_y))

  ridgeCv = RidgeClassifierCV().fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", ridgeCv.score(minmaxtest_X, test_y))
  ridgeCv = RidgeClassifierCV().fit(StdTrain_X, train_y)
  print("StandardScaler", ridgeCv.score(Stdtest_X, test_y))
  ridgeCv = RidgeClassifierCV().fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", ridgeCv.score(maxAbstest_X, test_y))
  ridgeCv = RidgeClassifierCV().fit(robustTrain_X, train_y)
  print("RobustScaler", ridgeCv.score(robusttest_X, test_y))
  print()

  from sklearn.linear_model import SGDClassifier
  sgd = SGDClassifier().fit(train_X, train_y)
  print("SGDClassifier",sgd.score(test_X, test_y))

  sgd = SGDClassifier().fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", sgd.score(minmaxtest_X, test_y))
  sgd = SGDClassifier().fit(StdTrain_X, train_y)
  print("StandardScaler", sgd.score(Stdtest_X, test_y))
  sgd = SGDClassifier().fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", sgd.score(maxAbstest_X, test_y))
  sgd = SGDClassifier().fit(robustTrain_X, train_y)
  print("RobustScaler", sgd.score(robusttest_X, test_y))
  print()

  from sklearn import svm
  svc = svm.SVC(gamma='scale').fit(train_X, train_y)
  print("SVC",svc.score(test_X, test_y))

  svc = svm.SVC(gamma='scale').fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", svc.score(minmaxtest_X, test_y))
  svc = svm.SVC(gamma='scale').fit(StdTrain_X, train_y)
  print("StandardScaler", svc.score(Stdtest_X, test_y))
  svc = svm.SVC(gamma='scale').fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", svc.score(maxAbstest_X, test_y))
  svc = svm.SVC(gamma='scale').fit(robustTrain_X, train_y)
  print("RobustScaler", svc.score(robusttest_X, test_y))
  print()

  from sklearn.neural_network import MLPClassifier
  mlp = MLPClassifier(random_state=411).fit(train_X, train_y)
  print("MLPClassifier", mlp.score(test_X, test_y))

  mlp = MLPClassifier(random_state=411).fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", mlp.score(minmaxtest_X, test_y))
  mlp = MLPClassifier(random_state=411).fit(StdTrain_X, train_y)
  print("StandardScaler", mlp.score(Stdtest_X, test_y))
  mlp = MLPClassifier(random_state=411).fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", mlp.score(maxAbstest_X, test_y))
  mlp = MLPClassifier(random_state=411).fit(robustTrain_X, train_y)
  print("RobustScaler", mlp.score(robusttest_X, test_y))
  print()

  mlp = MLPClassifier(hidden_layer_sizes=[70], max_iter=300, random_state=411).fit(train_X, train_y)
  print("MLPClassifier",mlp.score(test_X, test_y))

  mlp = MLPClassifier(hidden_layer_sizes=[70], max_iter=300, random_state=411).fit(minmaxTrain_X, train_y)
  print("MinMaxScaler", mlp.score(minmaxtest_X, test_y))
  mlp = MLPClassifier(hidden_layer_sizes=[70], max_iter=300, random_state=411).fit(StdTrain_X, train_y)
  print("StandardScaler", mlp.score(Stdtest_X, test_y))
  mlp = MLPClassifier(hidden_layer_sizes=[70], max_iter=300, random_state=411).fit(maxAbsTrain_X, train_y)
  print("MaxAbsScaler", mlp.score(maxAbstest_X, test_y))
  mlp = MLPClassifier(hidden_layer_sizes=[70], max_iter=300, random_state=411).fit(robustTrain_X, train_y)
  print("RobustScaler", mlp.score(robusttest_X, test_y))

  from xgboost import XGBClassifier

  xgb = XGBClassifier(n_estimators =500, random_state=411)

  eval_set = [(train_X, train_y), (test_X, test_y)] 
  eval_metric = ["auc", "error"]
  xgb.fit(train_X, train_y, eval_metric=eval_metric, eval_set=eval_set, verbose=False)
  print("XGboost :", xgb.score(test_X, test_y))

  xgb = XGBClassifier(n_estimators =500, random_state=411)
  eval_set = [(minmaxTrain_X, train_y), (minmaxtest_X, test_y)] 
  xgb.fit(minmaxTrain_X, train_y, eval_metric=eval_metric, eval_set=eval_set, verbose=False)
  print("XGboost :", xgb.score(minmaxtest_X, test_y))

  xgb = XGBClassifier(n_estimators =500, random_state=411)
  eval_set = [(StdTrain_X, train_y), (Stdtest_X, test_y)] 
  xgb.fit(StdTrain_X, train_y, eval_metric=eval_metric, eval_set=eval_set, verbose=False)
  print("XGboost :", xgb.score(Stdtest_X, test_y))

  xgb = XGBClassifier(n_estimators =500, random_state=411)
  eval_set = [(maxAbsTrain_X, train_y), (maxAbstest_X, test_y)] 
  xgb.fit(maxAbsTrain_X, train_y, eval_metric=eval_metric, eval_set=eval_set, verbose=False)
  print("XGboost :", xgb.score(maxAbstest_X, test_y))

  xgb = XGBClassifier(n_estimators =500, random_state=411)
  eval_set = [(robustTrain_X, train_y), (robusttest_X, test_y)] 
  xgb.fit(robustTrain_X, train_y, eval_metric=eval_metric, eval_set=eval_set, verbose=False)
  print("XGboost :", xgb.score(robusttest_X, test_y))







  print("\n\n\n")



PassiveAggressiveClassifier 0.3687150837988827
MinMaxScaler 0.776536312849162
StandardScaler 0.7039106145251397
MaxAbsScaler 0.7988826815642458
RobustScaler 0.7597765363128491

RidgeClassifier 0.7988826815642458
MinMaxScaler 0.7988826815642458
StandardScaler 0.7988826815642458
MaxAbsScaler 0.7988826815642458
RobustScaler 0.7988826815642458

RidgeClassifierCV 0.8044692737430168
MinMaxScaler 0.7988826815642458
StandardScaler 0.7988826815642458
MaxAbsScaler 0.7988826815642458
RobustScaler 0.8044692737430168

SGDClassifier 0.7821229050279329
MinMaxScaler 0.8044692737430168
StandardScaler 0.7486033519553073
MaxAbsScaler 0.7821229050279329
RobustScaler 0.7821229050279329

SVC 0.7318435754189944
MinMaxScaler 0.8491620111731844
StandardScaler 0.8379888268156425
MaxAbsScaler 0.8268156424581006
RobustScaler 0.8379888268156425



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier 0.8379888268156425


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MinMaxScaler 0.8435754189944135


...

StandardScaler 0.8321678321678322


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MaxAbsScaler 0.8391608391608392


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


RobustScaler 0.8251748251748252
XGboost : 0.7902097902097902
XGboost : 0.7902097902097902
XGboost : 0.7902097902097902
XGboost : 0.7902097902097902
XGboost : 0.7902097902097902






In [97]:
# total dataset Train for release 
totalSet = processed_df[0]
label = list(totalSet.columns)

label.remove('Survived')


X = totalSet[label]
std = StandardScaler().fit(X)
std_X = std.transform(X)
y = totalSet['Survived']
mlp = MLPClassifier(hidden_layer_sizes=[70], max_iter=300, random_state=411).fit(std_X, y)
print()



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [98]:
# prediction 
# feature_label.remove("Survived")
test = test_df[feature_label]
test = dataProcessing(test)
test = oneHotEncoding(test)

testX = test[0]
print(testX.shape)
std_testX = std.fit_transform(testX)
print(std_testX.shape)




(418, 10)
(418, 10)


In [99]:
predict = mlp.predict(std_testX)
concat = predict.reshape(-1,1)
test = test_df["PassengerId"]
result = pd.concat([test, pd.DataFrame(concat, columns=["Survived"])], axis=1)


result.to_csv("result.csv", index=False)


In [203]:
# import torch
# import torchvision 
# import torch.nn as nn

# class Model(nn.Module):
#   def __init__(self, input_size, channel, output):
#     super(Model, self).__init__()

#     self.mlp = nn.Seqential(
        
#         nn.lienar
#     )
